<a href="https://colab.research.google.com/github/hyomee2/sswu-lc5500-aiapp/blob/main/Llama/models/llama_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## 1. github 클론

In [ ]:
! git clone https://github.com/hyomee2/sswu-lc5500-aiapp.git

In [ ]:
!cd sswu-lc5500-aiapp

In [ ]:
import pandas as pd
from datasets import Dataset
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

## 2. 데이터셋 불러와서 훈련 포맷으로 맞추기

In [ ]:
df = pd.read_csv("/content/sswu-lc5500-aiapp/Llama/dataset/Llama_data.csv").dropna(subset=["Q", " A"]) # 결측치 제외 후 불러오기

# JSON 리스트 만들기
records = []
for _, row in df.iterrows():
    records.append({
        "instruction": row["Q"].strip(), # 질문 앞뒤 공백 제거
        "input": "", # input은 비움
        "output": str(row[" A"]).strip() # A 앞에 들어있는 공백 제거
    })

# JSON 저장
with open("/content/gdrive/MyDrive/aiapplication/train_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

## 3. Huggingface 데이터셋으로 변환 및 로딩

In [ ]:
# JSON을 pandas로 읽기
df = pd.read_json("/content/gdrive/MyDrive/aiapplication/train_data.json")

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# train/validation 90:10으로 나누기
split_dataset = dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

## 4. HuggingFace 모델 불러오기

In [ ]:
!pip install transformers accelerate peft bitsandbytes datasets

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hugging_face_token = userdata.get("HUGGING_FACE_TOKEN")
login(hugging_face_token)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_name = "beomi/Llama-3-Open-Ko-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token  # pad_token 설정
tokenizer.padding_side = "left" # Llama는 autoregressive 모델이어서 오른쪽부터 단어를 예측하므로, 왼쪽 padding이 자연스럽다.

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_8bit=True, # LoRA를 위해 8bit 로딩
    device_map="auto"
)

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# LoRA를 위한 준비
model = prepare_model_for_kbit_training(model)

# LoRA 설정
lora_config = LoraConfig(
    r=16, # LoRA의 랭크(저차원 행렬의 크기). 보통 4~16 사이로 설정
    lora_alpha=32, # LoRA 내 스케일링 팩터. 학습 안정성에 도움
    target_modules=["q_proj", "v_proj"],  # LoRA가 어떤 신경망 내부 레이어(모듈)에만 적용될지 지정
    lora_dropout=0.05, # 과적합 방지를 위해 LoRA 적용 시 드롭아웃 확률
    bias="none", # bias 파라미터 업데이트 여부(none, all, lora 등)
    task_type="CAUSAL_LM" # 작업 유형 지정. 여기서는 언어 생성용 인과적 언어모델
)

model = get_peft_model(model, lora_config)


*** target_modules=["q_proj", "v_proj"]**

**Q, K, V란?**

Transformer 모델의 핵심인 어텐션(attention) 계산에서
Q: Query (질문)
K: Key (키)
V: Value (값)

이 세 개 행렬(벡터)을 만들어서 어텐션 점수를 계산.


**"q_proj", "v_proj"는 무엇인가?**

Transformer 구현체마다 다르지만, 보통 Q, K, V를 만들 때
q_proj, k_proj, v_proj라는 이름으로 각각의 선형 변환(Linear layer, 즉 행렬곱)을 만든다.

e.g.,
``` python
self.q_proj = nn.Linear(hidden_dim, hidden_dim)
self.k_proj = nn.Linear(hidden_dim, hidden_dim)
self.v_proj = nn.Linear(hidden_dim, hidden_dim)
```


**그럼 target_modules=["q_proj", "v_proj"]는?**

LoRA를 Q 프로젝션 레이어와 V 프로젝션 레이어에만 적용하겠다는 의미. 즉, Q와 V를 만드는 선형 변환에 LoRA 어댑터가 붙어서 학습된다. K는 빼고 Q, V에만 적용하는 건 실험적으로 성능이나 효율이 좋다고 알려진 경우가 많다. Q는 모델이 "무엇을 집중할지"를 결정하는 쿼리 정보고, V는 실제로 어텐션에서 참고하는 값이라, 이 두 부분을 조정하는 것이 모델 미세조정에 효과적일 때가 많다. K는 상대적으로 업데이트 효과가 적거나 중복될 수 있어서 제외하기도 한다.





## 5. 텍스트를 LLM 학습 포맷으로 변환

In [ ]:
# Prompt 템플릿 함수
def format_prompt(example):
    return f"""### 질문:\n{example['instruction']}\n\n### 답변:\n{example['output']}"""

**format_prompt(example)**

데이터를 LLM에 학습시키기 좋은 문자열 형태(Prompt)로 만들어준다.

e.g.,

```
### 질문:

12시 땡이다!

### 답변:

하루 또 가뿌네.
```

In [ ]:
# 토크나이징
def tokenize(example):
    prompt = format_prompt(example)
    return tokenizer(prompt, padding=True, truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize, remove_columns=val_dataset.column_names)

**tokenize()**

위에서 만든 문자열 prompt을 모델 학습용 숫자 토큰들로 변환한다.

padding="max_length": 길이가 512보다 짧으면 빈 자리를 채운다.

truncation=True: 너무 길면 512까지만 자름

결과는 다음과 같은 dict 형태

e.g.,
```python
{
  'input_ids': [...숫자들...],
  'attention_mask': [...1, 1, 1, 0, 0, 0...]  # pad된 부분은 0
}
```




## 6. 모델 학습을 위한 collator(데이터 배치 준비 도우미) 준비

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# min=False -> GPT,LLAMA 같은 casual language model 학습에 적합

## 7. Trainer 설정

**TrainingAruguments**

: 모델 학습과 관련된 다양한 하이퍼파라미터와 설정들을 모아놓은 객체. Trainer에게 어떻게 학습할지 알려주는 설정 모음집이라 할 수 있다.

In [ ]:
from transformers import TrainingArguments, Trainer

# Trainer 설정
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/models/checkpoints", # 학습 체크포인트(모델 가중치 등)를 저장할 폴더 경로
    per_device_train_batch_size=2, # 학습 시 GPU 1대당 사용할 배치 사이즈
    per_device_eval_batch_size=4, # 평가 시 GPU 1대당 사용할 배치 사이즈
    gradient_accumulation_steps=8,
    num_train_epochs=3, # 총 학습 Epoch 수(3번 데이터셋을 반복 학습)
    logging_dir="./logs", # 로그(학습 진행상황 등)를 저장할 디렉토리
    learning_rate=5e-5,  # 학습률 설정
    weight_decay=0.01,  # L2 정규화
    lr_scheduler_type="linear",  # 학습률 스케줄러
    logging_steps=10, # 몇 스텝마다 로그를 기록할지(여기선 10스텝마다)
    save_strategy="steps", # 체크포인트 저장 주기(여기선 step마다 저장)
    #evaluation_strategy="epoch", # 평가 주기(여기선 epoch마다 평가)
    fp16=True, # 16-bit half precision 사용 여부(속도 향상 및 메모리 절약)
    save_steps=50,
    save_total_limit=2, # 저장할 체크포인트 최대 개수(초과 시 오래된 것부터 삭제)
    report_to="wandb"  # WandB에 결과 보고
)

trainer = Trainer(
    model=model, # 학습할 PyTorch 모델
    args=training_args, # TrainingArguments 객체(학습 설정)
    train_dataset=tokenized_train, # 학습용 데이터셋
    eval_dataset=tokenized_val, # 평가용 데이터셋
    tokenizer=tokenizer, # 토크나이저(토큰 ID 변환 및 디코딩 등에 사용)
    data_collator=data_collator, # 배치 데이터를 만들 떄 자동으로 padding 등을 처리하는 함수
)

In [ ]:
import torch

# torch.cuda.empty_cache()

# 학습 시작
# trainer.train(resume_from_checkpoint=True)
# wanb.init
trainer.train()

# 모델 저장
model.save_pretrained("/content/gdrive/finetuned-kollama-dialect")
tokenizer.save_pretrained("/content/gdrive/finetuned-kollama-dialect")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/gdrive/MyDrive/finetuned_kollama_dialect"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()  # 평가 모드로 전환 (추론할 때)


In [ ]:
prompt = "User: 덥긴한데 벌써부터 에어컨 틀면 전기세가 느무 많이 나오겠제?\nBot:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [ ]:
import torch
import re
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=60,         # 너무 길게 생성 못하도록 제한
        min_length=10,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        eos_token_id=tokenizer.eos_token_id,  # EOS에서 멈춤
    )

full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 프롬프트 이후의 응답만 추출
response = full_text[len(prompt):].strip().split("\nUser")[0].strip()
sentences = re.findall(r'[^.!?]*[.!?]', response, re.UNICODE)

# 양쪽 공백 제거
clean_sentences = [s.strip() for s in sentences]

# 완성된 문장들만 출력
for sentence in clean_sentences:
    print(sentence)

In [ ]:
!pip install gTTS

In [ ]:
from gtts import gTTS
from IPython.display import Audio

if clean_sentences:
    tts = gTTS(text=response, lang='ko')
    tts.save("output.mp3")
    display(Audio("output.mp3", autoplay=True))
else:
    print("생성된 문장이 없습니다.")